In [98]:
import pandas as pd
import numpy as np
import os

file_locations = {
    'tpch': ['customer/orders_customer_optimal_rep.csv', 'orders/customer_orders_optimal_rep.csv', 
             'orders/lineitem_orders_optimal_rep.csv', 'part/lineitem_part_optimal_rep.csv', 
             'part/partsupp_part_optimal_rep.csv', 'partsupp/lineitem_partsupp_optimal_rep.csv', 
             'partsupp/part_partsupp_optimal_rep.csv', 'partsupp/supplier_partsupp_optimal_rep.csv', 
             'supplier/lineitem_supplier_optimal_rep.csv', 'supplier/partsupp_supplier_optimal_rep.csv'],
    'imdb': ['cast_info/title_cast_info_optimal_rep.csv',
             'movie_companies/title_movie_companies_optimal_rep.csv', 'movie_info/title_movie_info_optimal_rep.csv', 
             'movie_info_idx/title_movie_info_idx_optimal_rep.csv', 'movie_keyword/title_movie_keyword_optimal_rep.csv',
             'title/cast_info_title_optimal_rep.csv', 'title/movie_companies_title_optimal_rep.csv',
             'title/movie_info_idx_title_optimal_rep.csv', 'title/movie_info_title_optimal_rep.csv',
             'title/movie_keyword_title_optimal_rep.csv'
            ],
    'ssb': ['customer/lineorder_customer_optimal_rep.csv', 'ddate/lineorder_ddate_optimal_rep.csv', 
           'part/lineorder_part_optimal_rep.csv', 'supplier/lineorder_supplier_optimal_rep.csv'
           ]
}


datasets = ['ssb', 'tpch', 'imdb']



dfs = []


for d in datasets:
    for f in file_locations[d]:
        dfs.append(pd.read_csv(os.path.join("../data/", d, f)))

ds = pd.concat(dfs)
ds

,left_cardinality,left_cardinality_ratio,base_cardinality,selectivity_on_indexed_attr,left_ordered,base_ordered,result_size,sel_on_indexed_attr_with_join_predicate,hj_idx_cost,hj_seq_cost,nl_idx_cost,nl_seq_cost,mj_idx_cost,mj_seq_cost,optimal_decision
0,10800,0.360000,30000,0.908900,0,1,9816,0.454467,3078.65,2821.50,5701.00,4.418208e+06,2197.32,3906.91,4
1,3265,0.108833,30000,0.671800,0,1,2193,0.335900,1188.48,1172.10,3383.99,9.879785e+05,1207.18,2656.47,3
2,4860,0.162000,30000,0.562533,0,1,2734,0.281267,1002.33,1142.52,3874.45,1.231189e+06,1170.05,2504.54,2
3,7892,0.263067,30000,0.692467,0,1,5465,0.346233,1237.22,1229.92,4806.79,2.460164e+06,1613.73,3084.59,3
4,5875,0.195833,30000,0.308067,0,1,1810,0.154033,557.97,1041.66,4186.57,8.153604e+05,865.39,1956.25,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,249337,0.098618,2528312,0.481484,1,1,120056,0.240742,110942.47,123540.02,292249.65,8.607989e+09,89437.53,352096.40,4
1996,630771,0.249483,2528312,0.852889,1,1,537998,0.426444,198809.96,173656.13,463894.95,3.857394e+10,187256.88,610160.72,3
1997,314278,0.124303,2528312,0.956698,1,1,300681,0.478349,217297.85,176963.25,321473.10,2.155971e+10,157455.99,624699.01,4
1998,778601,0.307953,2528312,0.788704,1,1,614109,0.394352,186692.35,170107.88,530418.45,4.403258e+10,199352.37,594840.84,3


In [99]:
ds.columns

Index(['left_cardinality', 'left_cardinality_ratio', 'base_cardinality',
       'selectivity_on_indexed_attr', 'left_ordered', 'base_ordered',
       'result_size', 'sel_on_indexed_attr_with_join_predicate', 'hj_idx_cost',
       'hj_seq_cost', 'nl_idx_cost', 'nl_seq_cost', 'mj_idx_cost',
       'mj_seq_cost', 'optimal_decision'],
      dtype='object')

# Classification

### Complex model + full features

In [100]:
# key_features = ['left_cardinality_ratio', 'selectivity_on_indexed_attr', 'left_ordered', 'base_ordered']

features = ['left_cardinality', 'left_cardinality_ratio', 'base_cardinality',
       'selectivity_on_indexed_attr', 'left_ordered', 'base_ordered',
       'result_size', 'sel_on_indexed_attr_with_join_predicate']

regression_targets = ['hj_idx_cost', 'hj_seq_cost', 'nl_idx_cost', 'nl_seq_cost','mj_idx_cost', 'mj_seq_cost']
classification_target = ['optimal_decision']


X = ds[features]
y = ds[classification_target]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

# scaler = preprocessing.StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)




/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [101]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

clf = MLPClassifier(random_state=1, hidden_layer_sizes=(100,100), max_iter=3000).fit(X_train, y_train)

print(f'Train accuracy: {clf.score(X_train, y_train)}')
print(f'Test accuracy: {clf.score(X_test, y_test)}')

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train accuracy: 0.9688611111111111
Test accuracy: 0.9690833333333333


### Complex model + selected features

# Regression

### hash join + index scan

In [102]:
# key_features = ['left_cardinality_ratio', 'selectivity_on_indexed_attr', 'left_ordered', 'base_ordered']

features = ['left_cardinality', 'left_cardinality_ratio', 'base_cardinality',
       'selectivity_on_indexed_attr', 'left_ordered', 'base_ordered',
       'result_size', 'sel_on_indexed_attr_with_join_predicate']

regression_targets = ['hj_idx_cost', 'hj_seq_cost', 'nl_idx_cost', 'nl_seq_cost','mj_idx_cost', 'mj_seq_cost']
classification_target = ['optimal_decision']


X = ds[features]
y = ds[regression_targets[0]]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

# scaler = preprocessing.StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [103]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

regr = MLPRegressor(random_state=1, hidden_layer_sizes=(100, 100,100 ,100, 100), max_iter=300).fit(X_train, y_train)

loss_in_percentage = np.average(np.abs(regr.predict(X_test) - y_test)) / np.average(y_test)
print(f"Loss in percentage: +- {loss_in_percentage * 100}%")

Loss in percentage: +- 1.8231766802139493%


### hash join + Seq scan

In [ ]:
# key_features = ['left_cardinality_ratio', 'selectivity_on_indexed_attr', 'left_ordered', 'base_ordered']

features = ['left_cardinality', 'left_cardinality_ratio', 'base_cardinality',
       'selectivity_on_indexed_attr', 'left_ordered', 'base_ordered',
       'result_size', 'sel_on_indexed_attr_with_join_predicate']

regression_targets = ['hj_idx_cost', 'hj_seq_cost', 'nl_idx_cost', 'nl_seq_cost','mj_idx_cost', 'mj_seq_cost']
classification_target = ['optimal_decision']


X = ds[features]
y = ds['hj_seq_cost']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

# scaler = preprocessing.StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

regr = MLPRegressor(random_state=1, hidden_layer_sizes=(100, 100,100 ,100, 100), max_iter=300).fit(X_train, y_train)

loss_in_percentage = np.average(np.abs(regr.predict(X_test) - y_test)) / np.average(y_test)
print(f"Loss in percentage: +- {loss_in_percentage * 100}%")